# Нормазизация. BatchNormalization

**Нормализация данных** - один из способов ускорить обучение нейронной сети. На левом графике показаны сырые данные (для задачи регрессии), которые далеко находятся от начальных весов сети. Чтобы приблизиться к данным, должно быть совершено достаточно большое количество шагов градиентного спуска. если данные будут, нормализованы, тогда количество шагов градиентного спуска заметно уменьшиться (правый график), что позволит ускорить и стабилизировать обучение нейронной сети.

![](https://raw.githubusercontent.com/TheDim0n/TensorFlow-Learning/master/images/normalization.png)

Рассмотрим одну из разновидностей нормализации - **BatchNormalization** - нормализация по батчу (пакету данных). Пусть $ B = \{x_{1}, x_{2}, ..., x_{N}\}$ - батч, тогда нормализация по батчу:
$$ \Large BN_{\alpha, \beta}({X}_{i}) = \alpha\hat{X} + \beta,$$
где:
$ \Large \hat{X} = \frac{x_{i}-\mu_{B}}{\sqrt{\sigma^{2}_{B}+\epsilon}} $ $\Large-$ нормализация, <br><br>
$ \Large \sigma^{2}_{B} = \frac{1}{N} \sum _{i=1} ^{N} (x_{i}-\mu_{B})^{2} $ $\Large-$ дисперсия батча, <br><br>
$ \Large \mu_{B} = \frac{1}{N} \sum _{i=1} ^{N} x_{i} $ $\Large-$ математическое ожидание батча, <br><br>
$ \Large \alpha, \beta $ $\Large-$ обучаемые параметры слоя. <br><br>
Слой батч-нормализации можно применять не только ко входному слою (к сырым данным), но и к последующим слоям нейронной сети (признакам). Например:

In [12]:
tf.keras.Sequential([
    tf.keras.layers.InputLayer((28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 3, 1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, 1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, 3, 1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
], name='model').summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 32)          36896 

# Регуляризация. L1, L2 и Dropout
**Переобучение** модели нейронной сети - это ситуация, когда модель "выучила" тренировочные данные, однако закономерность в них не выявила. В таком случае метрики на тренировочном датасете будут высокими, а на тестовом - низкими.
Основные причины переобучения:
- слишком много параметров нейронной сети;
- мало данных (Imbalanced Data).

С первой причиной борются методы регуляризации. Рассмотрим их подробнее.

## L1-регуляризация
Допустим, мы обучаем нейронную сеть на функции ошибки **MSE** (может быть и любая другая):
$$ \Large MSE = \frac{1}{N} \sum _{i=1} ^{N} (y_{i}-\hat{y})^{2}$$
Давайте добавим к ней сумму модулей весов сети. Мы получили формулу L1-регуляризации (её также называют LASSO-регуляризацией):
$$ \Large Loss = MSE + \alpha \sum _{i=1} ^{M} |w_{i}|,$$
где: <br><br>
$ \Large M $ $\Large-$ количество обучаемых параметров модели, <br><br>
$ \Large w_{i} $ $\Large-$ $i$-ый вес модели. <br><br>
## L2-регуляризация
На этот раз добавим к функции потерь сумму квадратов весов. Мы получили формулу L2-регуляризации (её также называют регуляризацией Тихонова):
$$ \Large Loss = MSE + \alpha \sum _{i=1} ^{M} w_{i}^2,$$
где: <br><br>
$ \Large M $ $\Large-$ количество обучаемых параметров модели, <br><br>
$ \Large w_{i} $ $\Large-$ $i$-ый вес модели. <br><br>
В обоих методах $\Large \alpha$ $\Large-$ настраиваемый параметр.

## Dropout
Dropout, как и регуляризация, не позволяет весам нейронной сети принимать слишком большие (или слишком маленькие) значения. Метод заключается в обнулении определнного процента весов слоя (обычно это 30-50%) на время обучения - фактически урезается кол-во параметров слоя и ,как следствие, всей сети. На каждом шаге градиентного спуска обнуляемые веса выбираются случайным образом.

Пример реализации в Keras вышеописанных методов:

In [13]:
tf.keras.Sequential([
    tf.keras.layers.InputLayer((100,)),
    tf.keras.layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.L2(0.1)),
    tf.keras.layers.Dropout(0.5), # обнуляем 50% весов ТОЛЬКО ПРЕДЫДУЩЕГО СЛОЯ
    tf.keras.layers.Dense(10, activation='softmax')
], name='model').summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                10250     
Total params: 113,674
Trainable params: 113,674
Non-trainable params: 0
_________________________________________________________________


Документация по регуляризации и Dropout:<br>
L1 - https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L1 <br>
L2 - https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2 <br>
L1L2- https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L1L2<br>
Dropout - https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout <br>